# Imports

In [1]:
import functions as func
import datasets
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm

cut_off_date = '2016-01-01'

Using TensorFlow backend.
/home/benayas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/benayas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/benayas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/benayas/.local/lib/python3.6/site-packages/tensorflow/py

# Copiar datos desde GS
Ejecutar esta parte en local desde la carpeta en la que esta "data/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'clean-composite-253713'
!gcloud config set project {project_id}
!gsutil ls

In [ ]:
bucket_name = 'tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-0003*'
dest = '/content/drive/My Drive/Colab Notebooks/data'

!gsutil -m cp -r  gs://{bucket_name} '/content/drive/My Drive/Colab Notebooks/data'

# Copiar datos desde CSV a Feather
Ejecutar esta parte en local desde la carpeta en la que esta "data/"

In [ ]:
files = glob.glob('/home/benayas/datasets/raw/*')
j = 0
start = 3
grouping = 3
for i in tqdm(range(start, len(files), grouping)):
    df = func.toDF_all(files[j:i], reb=False)
    j = i
    df.reset_index(drop=True).to_feather('/home/benayas/datasets/standard/file_' + str(i) + ('.feather'))

In [ ]:
from sklearn.model_selection import train_test_split
files = glob.glob('/home/benayas/datasets/standard/*')
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df['LABEL'] = df['LABEL'].apply(lambda x: 0 if x == -1 else x )
    df_test = df[df['DATE']>=cut_off_date]
    df = df[df['DATE']<cut_off_date]
    df = func.rebalance(df)
    df_train, df_validation = train_test_split(df, shuffle=True, stratify=df[['LABEL']], test_size=0.1)
    df_train.reset_index(drop=True).to_feather('/home/benayas/datasets/balanced_up/train/file_' + str(i) + ('.feather'))
    df_validation.reset_index(drop=True).to_feather('/home/benayas/datasets/balanced_up/validation/file_' + str(i) + ('.feather'))
    func.rebalance(df_test).reset_index(drop=True).to_feather('/home/benayas/datasets/balanced_up/test/file_' + str(i) + ('.feather'))
    i = i+1

In [2]:
from sklearn.model_selection import train_test_split
files = glob.glob('/home/benayas/datasets/standard/*')
#i=0
#for f in tqdm(files):
    
j = 0
start = 3
grouping = 3
for i in tqdm(range(start, len(files), grouping)):
    #df = func.toDF(f)
    df = func.toDF_all(files[j:i], reb=False)
    df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == -1 else 0 )
    df_test = df[df['DATE']>=cut_off_date]
    df = df[df['DATE']<cut_off_date]
    #df = func.rebalance(df)
    df_train, df_validation = train_test_split(df, shuffle=True, stratify=df[['LABEL']], test_size=0.1)
    func.rebalance(df_train).reset_index(drop=True).to_feather('/home/benayas/datasets/balanced_down/train/file_' + str(i) + ('.feather'))
    func.rebalance(df_validation).reset_index(drop=True).to_feather('/home/benayas/datasets/balanced_down/validation/file_' + str(i) + ('.feather'))
    df_train.reset_index(drop=True).to_feather('/home/benayas/datasets/full_down/train/file_' + str(i) + ('.feather'))
    df_validation.reset_index(drop=True).to_feather('/home/benayas/datasets/full_down/validation/file_' + str(i) + ('.feather'))
    df_test.reset_index(drop=True).to_feather('/home/benayas/datasets/full_down/test/file_' + str(i) + ('.feather'))
    j = i
    #i = i+1

100%|██████████| 19/19 [16:57<00:00, 53.56s/it]


In [3]:
from sklearn.model_selection import train_test_split
files = glob.glob('/home/benayas/datasets/standard/*')
df_test_final = pd.DataFrame()

j = 0
start = 3
grouping = 3
for i in tqdm(range(start, len(files), grouping)):
    df = func.toDF_all(files[j:i], reb=False)
    j = i
    df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == -1 else 0)
    df = df[df['DATE']>=cut_off_date]
    df_test_final = pd.concat([df_test_final,func.rebalance(df).reset_index(drop=True)], ignore_index=True)

df_test_final.reset_index(drop=True).to_feather('/home/benayas/datasets/balanced_down/test/file_0' + ('.feather'))


100%|██████████| 19/19 [03:46<00:00, 11.93s/it]


In [ ]:
df_test = func.toDF('/home/benayas/datasets/balanced_up/test/file_9999' + ('.feather'))
len(df_test)

In [ ]:
print('Dataset Test balanceado cantidad de 1:{0}'.format(str(len(df_test[df_test['LABEL']==1]))) )
print('Dataset Test balanceado cantidad de 0:{0}'.format(str(len(df_test[df_test['LABEL']==0]))) )

# Modificar Convertir Datos a Feather (train, validation, test)
Ejecutar esta parte en local desde la carpeta en la que esta "data/"

In [ ]:
import functions as func
import datasets
import pandas as pd
import numpy as np
import glob

In [ ]:
files = glob.glob('/home/benayas/datasets/raw*')
len(files)

In [ ]:
datasets.copy_feather(files, grouping=3, reb=False)

# Rebalancear Dataset
Ejecutar esta parte en local desde la carpeta en la que esta "data/"

In [ ]:
from tqdm import tqdm
suffix='simple_down'
path = '/home/benayas/datasets/'

In [ ]:
files = glob.glob(path + 'train_simple/*')
len(files)
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == -1 else 0)
    df = func.rebalance(df)
    df.reset_index(drop=True).to_feather(path +'train_' + suffix + '/file_' + str(i) + ('.feather'))
    i=i+1

In [ ]:
files = glob.glob(path + 'test_simple/*')
len(files)
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == -1 else 0)
    df = func.rebalance(df)
    df.reset_index(drop=True).to_feather(path +'test_' + suffix + '/file_' + str(i) + ('.feather'))
    i=i+1

In [ ]:
files = glob.glob(path + 'validation_simple/*')
len(files)
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == -1 else 0)
    df = func.rebalance(df)
    df.reset_index(drop=True).to_feather(path + 'validation_' + suffix + '/file_' + str(i) + ('.feather'))
    i=i+1

# Data Augmentation
Ejecutar esta parte en local desde la carpeta en la que esta "data/"

In [ ]:
def negativo(x):
    return 255-x

from random import randint
r = randint(0, 256)
def alteracion_random(x):
    (x+r)%255

In [ ]:
from tqdm import tqdm
import time 
suffix='simple_aug_up'
path = '/home/benayas/datasets/'

In [ ]:
f = alteracion_random
files = glob.glob(path + 'simple_up/train/*')
len(files)
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df.iloc[:,21:] = df.iloc[:,21:].apply(f, axis=0)
    df.to_feather(path + suffix + '/train' + '/file_aug' + str(int(time.time())) + ('.feather'))
    i=i+1

files = glob.glob(path + 'simple_up/validation/*')
len(files)
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df.iloc[:,21:] = df.iloc[:,21:].apply(f, axis=0)
    df.to_feather(path + suffix + '/validation' + '/file_aug' + str(int(time.time())) + ('.feather'))
    i=i+1

# Dataset LSTM

In [ ]:
path = '/home/benayas/datasets/lstm/price_label.csv'
df = func.toDF(path)
df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == -1 else 0)
df_test = df[df['DATE']>=cut_off_date]
df_train = df[df['DATE']<cut_off_date]
df_test.reset_index(drop=True).to_feather('/home/benayas/datasets/lstm/train/test' + '.feather')
df_train.reset_index(drop=True).to_feather('/home/benayas/datasets/lstm/test/train' + '.feather')